In [60]:
import mysql.connector
from mysql.connector import Error

## Initiate connection to sql server

In [71]:
## connect to database
def connect_to_database(host, user, password, database):

    '''
    Establish a connection with existing database
    '''
    connection = None
    try:
        connection = mysql.connector.connect(
            host = host,
            user = user,
            passwd = password,
            database = database
        )
        print("The connection successfully done")
    except Error as err:
        print("ERROR:\n%s" % err)
    return connection

def execute_query(connection, query, show_results=True):
    '''
    Executes a sql query and returns the results if set to True
    '''
    cursor = connection.cursor()
    results = None
    try:
        cursor.execute(query)
        if show_results:
            results = cursor.fetchall()
            
        warnings = cursor.fetchwarnings()
        rows = cursor.rowcount
        connection.commit()
        print('''Query successfully executed.
                warings: %s,
                rows: %d
        ''' % (warnings, rows))
        
    except Error as err:
        print("ERROR:\n%s" % err)

    return results

In [69]:
connection = connect_to_database(
    host = "localhost",
    user = "root",
    password = "Bora@1992",
    database = "join_us"
)
connection

The connection successfully done


In [73]:
query = "SELECT COUNT(*) FROM users;"
execute_query(connection, query, show_results=True)

Query successfully executed.
                warings: None,
                rows: 1
        


[(500,)]

## Find the oldest user in the database:

In [86]:
query = '''
SELECT 
    email_id,
    DATE_FORMAT(created_at, "%M %D %Y")
FROM users ORDER BY created_at LIMIT 1;    
'''
results = execute_query(connection,query)
results[0]

Query successfully executed.
                warings: None,
                rows: 1
        


('erichorton@hotmail.com', 'January 10th 1970')

## Find the month wise distribution of number of users:

In [89]:
query = '''
SELECT
    DATE_FORMAT(created_at, "%M") AS month,
    COUNT(*) AS count
FROM users
GROUP BY month
ORDER BY count DESC;
'''
results = execute_query(connection,query)
results

Query successfully executed.
                warings: None,
                rows: 12
        


[('July', 52),
 ('March', 48),
 ('June', 46),
 ('February', 45),
 ('January', 44),
 ('October', 41),
 ('November', 40),
 ('August', 40),
 ('December', 40),
 ('May', 37),
 ('April', 36),
 ('September', 31)]

Find all yahoo.com users:

## find the count for each domain

In [97]:
query = '''
SELECT
    CASE
        WHEN email_id LIKE "%@yahoo.com" THEN "yahoo"
        WHEN email_id LIKE "%@gmail.com" THEN "gmail"
        WHEN email_id LIKE "%@hotmail.com" THEN "hotmail"
        ELSE "others"
    END AS domain,
    COUNT(*) as count
FROM users 
GROUP BY domain
ORDER BY count DESC;
'''
results = execute_query(connection,query)
results

Query successfully executed.
                warings: None,
                rows: 4
        


[('others', 267), ('hotmail', 82), ('gmail', 76), ('yahoo', 75)]

In [98]:
connection.close()